# HRNet 

- https://arxiv.org/abs/1904.04514
- https://arxiv.org/abs/1908.07919
- https://github.com/HRNet/HRNet-Semantic-Segmentation
- https://www.youtube.com/watch?v=sIP3MrFWCpg

![alt text](HRNet.png)

![alt text](fusion.png)

![alt text](fusion2.png)

In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout,
                                     BatchNormalization, Activation, UpSampling2D,
                                     Conv2DTranspose, concatenate, multiply, Lambda,
                                    Add)

from tensorflow.keras import backend as kb
from tensorflow.keras.optimizers import Adam


In [2]:
def res_conv_block(inputs=None, n_filters=64, dropout_prob=0):
    
    """
    Residual Convolutional Block
    
    Arguments:
        inputs - input tensor usually image shape (x, x, 3)
        n_filters - number of filters for conv layers
        dropout_prob - dropout probability (added for model regularization)
        
    Performs 2 convolutional steps with kernel size (3,3) stride 1 followed by batch normalization and or dropout depending
    on parameter sepcifications and adds a residual connection
    
    Returns:
        conv_layer 
        
    """
    
    # first conv layer
    conv1 = Conv2D(n_filters, 3, padding='same')(inputs)
    conv1 = BatchNormalization(axis=-1)(conv1)
    conv1 = Activation('relu')(conv1)
    
    # 2nd conv layer
    conv2 = Conv2D(n_filters, 3, padding='same')(conv1)
    conv2 = BatchNormalization(axis=-1)(conv2)
    conv2 = Activation('relu')(conv2)
    
    # adding Dropout 
    if dropout_prob > 0:
        conv2 = Dropout(dropout_prob)(conv2)
    
    # residual connection
    res_connection = Conv2D(n_filters, 1, padding='same')(inputs)
    res_connection = BatchNormalization(axis=-1)(res_connection)
    
    res_conv = Add()([res_connection, conv2])
    res_conv = Activation('relu')(res_conv)
    
    return res_conv

In [3]:
def down_sample_conv(inputs=None, n_filters=64, strides=(2,2)):
    
    """
    Performs downsampling via strided convolutions instead of pooling
    """

    down_conv = Conv2D(n_filters, 3, strides, padding='same')(inputs)
    
    return down_conv

In [4]:
def up_sample_conv(inputs=None, n_filters=64):
    
    """
    performs upsampling using bilinear 1x1 convolutions
    """
    up_conv = Conv2DTranspose(n_filters, 3, strides=2, padding='same')(inputs)
    
    return up_conv

In [5]:
def HR_block_one(inputs=None, n_filters=64, dropout_prob=0):
    
    """
    creates output for the first block in the HR net
    
    """
    
    res_block_1_1 = res_conv_block(inputs, n_filters, dropout_prob)
    
    res_block_1_2 = res_conv_block(res_block_1_1, n_filters, dropout_prob)
    
    res_block_1_3 = res_conv_block(res_block_1_2, n_filters, dropout_prob)
    
    res_block_1_4 = res_conv_block(res_block_1_3, n_filters, dropout_prob)
    
    # residual high resolution stream
    res_block_1_5 = res_conv_block(res_block_1_4, n_filters, dropout_prob)
    
    # begining of lower dimensional/resolution stream
    res_block_2_0 = down_sample_conv(res_block_1_4, n_filters*2)
    
    
    return res_block_1_5, res_block_2_0

In [6]:
def HR_block_two(l1_input, l2_input, n_filters=64, dropout_prob=0):
    
    """
    creates output for the second block in the HR net
    """
    res_block_1_6 = res_conv_block(l1_input, n_filters, dropout_prob)
    res_block_2_1 = res_conv_block(l2_input, n_filters*2, dropout_prob)
    
    res_block_1_7 = res_conv_block(res_block_1_6, n_filters, dropout_prob)
    res_block_2_2 = res_conv_block(res_block_2_1, n_filters*2, dropout_prob)
    
    res_block_1_8 = res_conv_block(res_block_1_7, n_filters, dropout_prob)
    res_block_2_3 = res_conv_block(res_block_2_2, n_filters*2, dropout_prob)
    
    res_block_1_9 = res_conv_block(res_block_1_8, n_filters, dropout_prob)
    res_block_2_4 = res_conv_block(res_block_2_3, n_filters*2, dropout_prob)
    
    # fusion
    up_samp = up_sample_conv(res_block_2_4, n_filters)
    down_samp = down_sample_conv(res_block_1_9, n_filters*2)
    
    res_block_1_10 = res_conv_block(res_block_1_9, n_filters, dropout_prob)
    res_block_1_10 = Add()([up_samp, res_block_1_10])
    res_block_1_10 = Activation('relu')(res_block_1_10)
    
    res_block_2_5 = res_conv_block(res_block_2_4, n_filters*2)
    res_block_2_5 = Add()([down_samp, res_block_2_5])
    res_block_2_5 = Activation('relu')(res_block_2_5)
    
    
    # begining of lower dimensional/resolution stream
    down_samp = down_sample_conv(down_samp, n_filters*4)
    res_block_3_0 = down_sample_conv(res_block_2_4, n_filters*4)
    res_block_3_0 = Add()([down_samp, res_block_3_0])
    res_block_3_0 = Activation('relu')(res_block_3_0)
    
    return res_block_1_10, res_block_2_5, res_block_3_0
    
    

In [7]:
def HR_block_three(l1_input, l2_input, l3_input, n_filters=64, dropout_prob=0):
    
    """
    creates output for the third block in the HR net
    """
    
    res_block_1_11 = res_conv_block(l1_input, n_filters, dropout_prob)
    res_block_2_6 = res_conv_block(l2_input, n_filters*2, dropout_prob)
    res_block_3_1 = res_conv_block(l3_input, n_filters*4, dropout_prob)
    
    res_block_1_12 = res_conv_block(res_block_1_11, n_filters, dropout_prob)
    res_block_2_7 = res_conv_block(res_block_2_6, n_filters*2, dropout_prob)
    res_block_3_2 = res_conv_block(res_block_3_1, n_filters*4, dropout_prob)
    
    res_block_1_13 = res_conv_block(res_block_1_12, n_filters, dropout_prob)
    res_block_2_8 = res_conv_block(res_block_2_7, n_filters*2, dropout_prob)
    res_block_3_3 = res_conv_block(res_block_3_2, n_filters*4, dropout_prob)
    
    res_block_1_14 = res_conv_block(res_block_1_13, n_filters, dropout_prob)
    res_block_2_9 = res_conv_block(res_block_2_8, n_filters*2, dropout_prob)
    res_block_3_4 = res_conv_block(res_block_3_3, n_filters*4, dropout_prob)
    
    # fusion
    l1_2 = down_sample_conv(res_block_1_14, n_filters*2)
    l1_3 = down_sample_conv(l1_2, n_filters*4)
    l1_4 = down_sample_conv(l1_3, n_filters*8)
    
    l2_1 = up_sample_conv(res_block_2_9, n_filters)
    l2_3 = down_sample_conv(res_block_2_9, n_filters*4)
    l2_4 = down_sample_conv(l2_3, n_filters*8)
    
    l3_2 = up_sample_conv(res_block_3_4, n_filters*2)
    l3_1 = up_sample_conv(l3_2, n_filters)
    l3_4 = down_sample_conv(res_block_3_4, n_filters*8)
    
    # building outputs
    res_block_1_15 = res_conv_block(res_block_1_14, n_filters, dropout_prob)
    res_block_1_15 = Add()([l2_1, l3_1, res_block_1_15])
    res_block_1_15 = Activation('relu')(res_block_1_15)
    
    res_block_2_10 = res_conv_block(res_block_2_9, n_filters*2, dropout_prob)
    res_block_2_10 = Add()([l1_2, l3_2,
                           res_block_2_10])
    res_block_2_10 = Activation('relu')(res_block_2_10)
    
    res_block_3_5 = res_conv_block(res_block_3_4, n_filters*4, dropout_prob)
    res_block_3_5 = Add()([l1_3, l2_3, res_block_3_5])
    res_block_3_5 = Activation('relu')(res_block_3_5)
    
    # begining of lower dimensional/resolution stream
    res_block_4_0 = Add()([l1_4, l2_4, l3_4])
    res_block_4_0 = Activation('relu')(res_block_4_0)
    
    return res_block_1_15, res_block_2_10, res_block_3_5, res_block_4_0
    

In [8]:
def HR_block_four(l1_input, l2_input, l3_input, l4_input, n_filters=64, dropout_prob=0):
    
    """
    creates output for the fourth block in the HR net
    """
    
    res_block_1_16 = res_conv_block(l1_input, n_filters, dropout_prob)
    res_block_2_11 = res_conv_block(l2_input, n_filters*2, dropout_prob)
    res_block_3_6 = res_conv_block(l3_input, n_filters*4, dropout_prob)
    res_block_4_1 = res_conv_block(l4_input, n_filters*8, dropout_prob)
    
    res_block_1_17 = res_conv_block(res_block_1_16, n_filters, dropout_prob)
    res_block_2_12 = res_conv_block(res_block_2_11, n_filters*2, dropout_prob)
    res_block_3_7 = res_conv_block(res_block_3_6, n_filters*4, dropout_prob)
    res_block_4_2 = res_conv_block(res_block_4_1, n_filters*8, dropout_prob)
    
    res_block_1_18 = res_conv_block(res_block_1_17, n_filters, dropout_prob)
    res_block_2_13 = res_conv_block(res_block_2_12, n_filters*2, dropout_prob)
    res_block_3_8 = res_conv_block(res_block_3_7, n_filters*4, dropout_prob)
    res_block_4_3 = res_conv_block(res_block_4_2, n_filters*8, dropout_prob)
    
    res_block_1_19 = res_conv_block(res_block_1_18, n_filters, dropout_prob)
    res_block_2_14 = res_conv_block(res_block_2_13, n_filters*2, dropout_prob)
    res_block_3_9 = res_conv_block(res_block_3_8, n_filters*4, dropout_prob)
    res_block_4_4 = res_conv_block(res_block_4_3, n_filters*8, dropout_prob)
    
    # fusion
    l1_2 = down_sample_conv(res_block_1_19, n_filters*2)
    l1_3 = down_sample_conv(l1_2, n_filters*4)
    l1_4 = down_sample_conv(l1_3, n_filters*8)
    
    l2_1 = up_sample_conv(res_block_2_14, n_filters)
    l2_3 = down_sample_conv(res_block_2_14, n_filters*4)
    l2_4 = down_sample_conv(l2_3, n_filters*8)
    
    l3_2 = up_sample_conv(res_block_3_9, n_filters*2)
    l3_1 = up_sample_conv(l3_2, n_filters)
    l3_4 = down_sample_conv(res_block_3_9, n_filters*8)
    
    l4_3 = up_sample_conv(res_block_4_4, n_filters*4)
    l4_2 = up_sample_conv(l4_3, n_filters*2)
    l4_1 = up_sample_conv(l4_2, n_filters)
    
    # building outputs
    res_block_1_20 = res_conv_block(res_block_1_19, n_filters, dropout_prob)
    res_block_1_20 = Add()([l2_1, l3_1, l4_1, res_block_1_20])
    res_block_1_20 = Activation('relu')(res_block_1_20)
    
    res_block_2_15 = res_conv_block(res_block_2_14, n_filters*2, dropout_prob)
    res_block_2_15 = Add()([l1_2, l3_2, l4_2, res_block_2_15])
    res_block_2_15 = Activation('relu')(res_block_2_15)
    
    res_block_3_10 = res_conv_block(res_block_3_9, n_filters*4, dropout_prob)
    res_block_3_10 = Add()([l1_3, l2_3, l4_3, res_block_3_10])
    res_block_3_10 = Activation('relu')(res_block_3_10)
    
    
    res_block_4_5 = res_conv_block(res_block_4_4, n_filters*8, dropout_prob)
    res_block_4_5 = Add()([l1_4, l2_4, l3_4, res_block_4_5])
    res_block_4_5 = Activation('relu')(res_block_4_5)
    
    
    return res_block_1_20, res_block_2_15, res_block_3_10, res_block_4_5

In [9]:
def classification_head(l1_input, l2_input, l3_input, l4_input, n_filters, n_classes):
    
    """
    Upsamples all the tiers of inputs to the l1_input shape,
    concatentates them together
    runs a 1 d convolutional operation with softmax activation for n_classes, returns semantic segmentation output
    """
    l1_1 = l1_input
    
    l2_1 = up_sample_conv(l2_input, n_filters)
    
    l3_2 = up_sample_conv(l3_input, n_filters*2)
    l3_1 = up_sample_conv(l3_2, n_filters)
    
    l4_3 = up_sample_conv(l4_input, n_filters*4)
    l4_2 = up_sample_conv(l4_3, n_filters*2)
    l4_1 = up_sample_conv(l4_2, n_filters)
    
    # final concatenation
    combined = Add()([l1_1, l2_1, l3_1, l4_1])
    combined = Activation('relu')(combined)
    
    # output tensor with number of channels based on number of target classes
    conv_out = Conv2D(n_classes, 1, padding='same', activation='softmax')(combined)
    
    return conv_out

In [10]:
def HRNet(input_size=(512,512,3), n_filters=64, n_classes=5):
    """
    
    """
    # input image
    inputs = Input(input_size)
    
    # block one
    l1_1, l2_1 = HR_block_one(inputs)
    
    # block two
    l1_2, l2_2, l3_1 = HR_block_two(l1_1, l2_1)
    
    # block three    
    l1_3, l2_3, l3_2, l4_1 = HR_block_three(l1_2, l2_2, l3_1)
    
    # block four
    l1_4, l2_4, l3_3, l4_2 = HR_block_four(l1_3, l2_3, l3_2, l4_1)
    
    # output
    output = classification_head(l1_4, l2_4, l3_3, l4_2, n_filters, n_classes)
    
    # create model object
    model = tf.keras.Model(inputs=inputs, outputs=output, name='HRNet')

    return model
    

In [13]:
img_shape = (512,512,3)
HRNet_model = HRNet(img_shape)
HRNet_model.summary()

Model: "HRNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_167 (Conv2D)             (None, 512, 512, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_150 (BatchN (None, 512, 512, 64) 256         conv2d_167[0][0]                 
__________________________________________________________________________________________________
activation_161 (Activation)     (None, 512, 512, 64) 0           batch_normalization_150[0][0]    
______________________________________________________________________________________________

In [14]:
# visualize the model
tf.keras.utils.plot_model(
    HRNet_model, to_file='model.png', show_shapes=False, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,
    layer_range=None
)